# Object tracking using Bayesian Tracker

This notebook shows how to previously localised and classified objects from a series of mask images and unite them over time using Bayesian multi-object tracking (btrack). 

The sections of this notebook are as follows:

1. Load segmentation objects and filter based on size
2. Run btrack, uniting the objects locations over time
3. Batch process

The data used in this notebook is timelapse microscopy data with h2b-gfp/rfp markers that show the spatial extent of the nucleus and it's mitotic state. 

In [1]:
import btrack
from tqdm import tqdm
import os
import glob

## 1. Load segmentation objects

In [2]:
expt = 'ND0011'
pos = 'Pos6'
root_dir = '/home/nathan/data'

In [ ]:
with btrack.dataio.HDF5FileHandler(
     f'{root_dir}/{expt}/{pos}/objects.h5', 'r', obj_type='obj_type_1',
) as hdf:
    objects_gfp = hdf.objects
with btrack.dataio.HDF5FileHandler(
     f'{root_dir}/{expt}/{pos}/objects.h5', 'r', obj_type='obj_type_2',
) as hdf:
    objects_rfp = hdf.objects

Check objects feature classification

In [ ]:
objects[0]

Excluding segments that are too small to feasibly be cells

In [109]:
objects = [o for o in objects if o.properties['area']>100.]

## 2. Run btrack  

Unite each object with it's subsequent position at the following time point and export as a tracking file. Note that you may need different btrack configurations for different populations.

#### For GFP objects

In [ ]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure_from_file(
        './models/MDCK_config_wildtype.json'
    )
    tracker.max_search_radius = 40

    # append the objects to be tracked
    tracker.append(objects_gfp)

    # set the volume
    tracker.volume=((0, 1200), (0, 1600), (-1e5, 1e5))

    # track them (in interactive mode)
    tracker.track_interactive(step_size=100)

    # generate hypotheses and run the global optimizer
    tracker.optimize()

    tracker.export(f'{root_dir}/{expt}/{pos}/tracks.h5', obj_type='obj_type_1')

    # get the tracks in a format for napari visualization (optional)
    visaulise_tracks, properties, graph = tracker.to_napari(ndim=2)
    
    gfp_tracks = tracker.tracks

#### For RFP objects

In [ ]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure_from_file(
        './models/MDCK_config_scribble_sparse.json'
    )
    tracker.max_search_radius = 40

    # append the objects to be tracked
    tracker.append(objects_rfp)

    # set the volume
    tracker.volume=((0, 1200), (0, 1600), (-1e5, 1e5))

    # track them (in interactive mode)
    tracker.track_interactive(step_size=100)

    # generate hypotheses and run the global optimizer
    tracker.optimize()

    tracker.export(f'{root_dir}/{expt}/{pos}/tracks.h5', obj_type='obj_type_2')

    # get the tracks in a format for napari visualization (optional)
    visaulise_tracks, properties, graph = tracker.to_napari(ndim=2)
    
    rfp_tracks = tracker.tracks

# 6. Batch process
Iterate over many experiments and positions (need to ensure you define normalisation and classification functions above first)

In [ ]:
root_dir = '/home/nathan/data'
expt_list = ['ND0009', 'ND0010', 'ND0011']
pos_list = 'all'
overwrite = False

for expt in expt_list:
    
        # Find all positions in that experiment, if pos_list is all then it finds all positions
        if pos_list == 'all':
            pos_list = [pos for pos in os.listdir(f'{root_dir}/{expt}') 
                    if 'Pos' in pos 
                    and os.path.isdir(f'{root_dir}/{expt}/{pos}')]  
            
        ### Iterate over all positions in that experiment
        for pos in pos_list:
            
            ### check if overwrite param is false check if raw directory already created and if type of transform file already exists and decide whether to skip pos
            if not overwrite and glob.glob(f'{root_dir}/{expt}/{pos}/*tracks*.h5'):
                print(glob.glob(f'{root_dir}/{expt}/{pos}/*tracks*.h5'), f'file found, skipping {expt}/{pos}')
                continue
                
            print(f'Starting {expt}/{pos}')
            # load gfp objects
            with btrack.dataio.HDF5FileHandler(
                 f'{root_dir}/{expt}/{pos}/objects.h5', 'r', obj_type='obj_type_1',
            ) as hdf:
                objects_gfp = hdf.objects
            # load rfp objects
            with btrack.dataio.HDF5FileHandler(
                 f'{root_dir}/{expt}/{pos}/objects.h5', 'r', obj_type='obj_type_1',
            ) as hdf:
                objects_rfp = hdf.objects
            
            # filter objects for non-cell type sizes
            objects_gfp = [o for o in objects_gfp if o.properties['area']>100.]
            objects_rfp = [o for o in objects_rfp if o.properties['area']>100.]
            
            # initialise a tracker session using a context manager for gfp
            with btrack.BayesianTracker() as tracker:

                # configure the tracker using a config file
                tracker.configure_from_file(
                    './models/MDCK_config_wildtype.json'
                )
                tracker.max_search_radius = 40

                # append the objects to be tracked
                tracker.append(objects_gfp)

                # set the volume
                tracker.volume=((0, 1200), (0, 1600), (-1e5, 1e5))

                # track them (in interactive mode)
                tracker.track_interactive(step_size=100)

                # generate hypotheses and run the global optimizer
                tracker.optimize()

                tracker.export(f'{root_dir}/{expt}/{pos}/tracks.h5', obj_type='obj_type_1')

            # initialise a tracker session using a context manager for rfp
            with btrack.BayesianTracker() as tracker:

                # configure the tracker using a config file
                tracker.configure_from_file(
                    './models/.json'
                )
                tracker.max_search_radius = 40

                # append the objects to be tracked
                tracker.append(objects_rfp)

                # set the volume
                tracker.volume=((0, 1200), (0, 1600), (-1e5, 1e5))

                # track them (in interactive mode)
                tracker.track_interactive(step_size=100)

                # generate hypotheses and run the global optimizer
                tracker.optimize()

                tracker.export(f'{root_dir}/{expt}/{pos}/tracks.h5', obj_type='obj_type_2')
                
            print(f'Finished {expt}/{pos}')